# XGBoost


## Introdução

<p align = "justify">O XGboost (Extreme Gradient Boosting) é um algoritmo de Ensemble, podendo ser pensado como uma aprimoração do Gradient Boosting. O Gradient Boosting ajusta os pesos usando o Gradient Descent (em geral olhando para uma loss function específica: MSE para algoritmos de regressão e uma função logarítmica para algoritmos de classificação), otimizando iterativamente a diferença entre o valor previsto e o valor real. Em essência, o XGBoost é um algoritmo que usa a expansão de Taylor de ordem 2 para tratar a função objetivo, tomando árvores de decisão como seus preditores. Além disso, a função objetivo do XGBoost conta com um parâmetro extra de regularização em sua construção, o qual aumenta a precisão, penaliza modelos muito complexos em detrimento de modelos mais simples e evita o temido sobreajuste (ideia que lembra muito a Ridge Regression).

## Fundamentação Teórica


<p align = "justify"> Seja $\mathcal{D}=\{ (x_i,y_i)_{i=1}^{n}\}⊂\mathbb{R}^m \times \mathbb{R}$ uma amostra rotulada. Um modelo emsemble de árvore de decisão usa um número $K$ de funções aditivas $f_k$ para predizer o parâmetro $y_i$. Essa predição é denotada por $p_i$ e </p>

$$p_i = \sum_{k=1}^{K} f_k(x_i),\qquad  f_k ∈ \mathcal{F},$$

em que $\mathcal{F} = \{f \mid f(x) = w_{q(x)},\, q:\mathbb{R}^m\to T \text{ e } w \in \mathbb{R}^T\}$

<p align = "justify"> Cada $f_k$ corresponde a uma estrutura de árvore independente $q$ e pesos de folha $w$, com $T$ sendo o número de folhas da árvore. Ao contrário das árvores de decisão, cada árvore de regressão contém uma pontuação contínua em cada folha: os $w_i$'s são usados para representar a pontuação na $i$-ésima folha. Para cada $x_i$, usaremos as regras de decisão nas árvores (dadas por $q$) para classificá-lo nas folhas e calcular a previsão final somando a pontuação nas folhas correspondentes (dadas por $w$). Para aprender o conjunto de funções usadas no modelo, se minimiza o objetivo regularizado: </p>

$$\mathcal{L} =  \sum_{i} l(p_i,y_i) +\sum_{k}\Omega(f_k)$$
onde $$\Omega(f) =\gamma T + \frac{1}{2}\lambda \|w\|^2.$$ 

<p align = "justify">
Aqui $l$ é uma $``$loss function$"$ (convexa e diferenciável) que mede a diferença entre a previsão $p_i$ e o alvo $y_i$. O segundo termo $\Omega$ penaliza a complexidade do modelo (ou seja, as funções da árvore de regressão). Esse termo de regularização ajuda a suavizar os pesos finais aprendidos para evitar sobreajuste (mesma ideia utilizada na construção do ridge regression visto em aula). Intuitivamente, o objetivo regularizado tenderá a selecionar um modelo empregando funções simples e preditivas. Quando o parâmetro de regularização é zerado, o objetivo volta ao tradicional Gboost. </p>

Seja $p_i^{(t)}$ a previsão da $i$-ésima instância na $t$-ésima iteração. Queremos encontrar $f_t$ para minimizar

$$\mathcal{L}^{(t)} =  \sum_{i= 1}^{n} l(y_i,p_i^{(t-1)}+f_t(x_i)) +\Omega(f_t)$$
Usando a expansão de Taylor de ordem 2, sabemos que 

$$f(a+h)≈ f(a)+f'(a)h+\frac{1}{2}f''(a)h^2$$ para $f\in C^2$. Se $l$ for suficientemente suave, fazemos $f=l$, $a = p_i^{(t)}$ e $h=f_t(x_i)$, obtendo assim

\begin{align}
\mathcal{L}^{(t)} &≈  \sum_{i= 1}^{n} \bigl( l(y_i,p_i^{(t-1)})+g_if_t(x_i)+\frac{1}{2}h_i [f_t(x_i)]^2\bigr) +\Omega(f_t) \\ &= \sum_{i= 1}^{n} \bigl[ C+g_if_t(x_i)+\frac{1}{2}h_i [f_t(x_i)]^2\bigr) +\Omega(f_t)
\end{align}
em que $C$ é uma constante, 
$$g_i = \frac{\partial l(y_i, p_i^{(t-1)})}{\partial p_i^{(t-1)}} \qquad \text{ e } \qquad h_i = \frac{\partial^2 l(y_i, p_i^{(t-1)})}{\partial p_i^{(t-1)}\,^2 }.$$

Removendo esse termo constante, nossa nova função objetivo se torna no passo $t$ igual a 

$$\mathcal{L_1}(t) = \Omega(f_t)+ \sum_{i= 1}^{n} \bigl(g_if_t(x_i)+\frac{1}{2}h_i [f_t(x_i)]^2\bigr).$$

Dado $j\in\{1,\dots, T\}$, defina $I_j  = \{i\mid q(x_i)=j\}$. Expandindo $\Omega$ e retomando definição de $\mathcal{F}$, podemos escrever
\begin{align}
\mathcal{L}_1^{(t)} &=\gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T} w_j^2+\sum_{i= 1}^{n} g_if_t(x_i)+\frac{1}{2}h_i [f_t(x_i)]^2 \\& = \gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T} w_j^2+ \sum_{j= 1}^{T}\sum_{i \in I_j} g_iw_j+\frac{1}{2}\sum_{j= 1}^{T}\sum_{i \in I_j} h_i[w_j]^2\\& = \gamma T + \sum_{j=1}^{T} \biggl[ \sum_{i \in I_j} g_iw_j+ \frac{1}{2}w_j^2\biggl( \lambda + \sum_{i \in I_j} h_i \biggr)\biggr]
\end{align}

Uma vez fixada a estrutura $q(x)$, podemos calcular para cada $j$ o peso ótimo $w_j^*$ dado por
$$
w_j^* = -\frac{\sum_{i \in I_j} g_i}{\lambda + \sum_{i \in I_j} h_i}
$$
<p align = "justify">(esse resultado pode ser obtido usando derivadas para encontrar os pontos críticos da parábola $ bx+\frac{a}{2}x^2$). Logo, o valor ótimo correspondente é </p>
$$
\mathcal{L}_1^{(t)}(q) = \gamma T-\frac{1}{2}\sum_{j=1}^{T}\frac{\bigl(\sum_{i \in I_j} g_i\bigr)^2}{\lambda + \sum_{i \in I_j} h_i}
$$

<p align = "justify">De modo geral, não é possível enumerar todas as estruturas de árvores $q$. Em vez disso, usa-se um algoritmo ambicioso que começa a partir de uma única folha e adiciona iterativamente ramos à árvore. Se $I_L$ e $I_R$ são os conjuntos de instâncias dos nós esquerdo e direito após a divisão, então deixando $I = I_L \cup I_R$ a redução da perda após a divisão é dada por</p>

\begin{align}
\mathcal{L}_{split} &= \gamma -\frac{1}{2}\frac{\bigl(\sum_{i \in I} g_i\bigr)^2}{\lambda + \sum_{i \in I} h_i} - \biggl[\gamma-\frac{1}{2}\frac{\bigl(\sum_{i \in I_L} g_i\bigr)^2}{\lambda + \sum_{i \in I_L} h_i} + \gamma -\frac{1}{2}\frac{\bigl(\sum_{i \in I_R} g_i\bigr)^2}{\lambda + \sum_{i \in I_R} h_i} \biggr]\\&=\frac{1}{2}\biggl[ \frac{\bigl(\sum_{i \in I_L} g_i\bigr)^2}{\lambda + \sum_{i \in I_L} h_i} +\frac{\bigl(\sum_{i \in I_R} g_i\bigr)^2}{\lambda + \sum_{i \in I_R} h_i} -\frac{\bigl(\sum_{i \in I} g_i\bigr)^2}{\lambda + \sum_{i \in I} h_i}\biggr]-\gamma.
\end{align}

## Prós e contras.



Prós:
*   Alta velocidade quando comparado com outros algorítmos.
*   Grande desempenho para dados tabulares/estruturados.
*   O termo de regularização do algorítmo penaliza a construção de árvores complexas, evitando o sobreajuste.
*   Pode lidar com valores ausentes.
*   Não há necessidade de dimensionar/normalizar dados.

Contras:
*   Não explora todas as estruturas de árvore possíveis.
*   Pode deixar a desejar se os dados não forem estruturados (ex. imagens).
*   Se os parâmetros da função objetivo não forem ajustados corretamente, pode facilmente sobreajustar os dados.


## Comparando o XGboost com outros algorítmos

In [18]:
import pandas as pd
import numpy as np
import os

In [19]:
seed = 42
np.random.seed(seed)

In [20]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

### Regressão - Dataset California Housing




In [21]:
housing = pd.read_csv("./sample_data/california_housing_train.csv")

In [22]:
X = housing.drop("median_house_value", axis=1)
Y = housing['median_house_value']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                        test_size=0.3, 
                                        random_state=seed)

In [23]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(max_features=8, n_estimators=30, random_state=seed)
forest_scores = cross_val_score(forest_reg, X_train, Y_train,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [48277.93131024 48843.55962478 53109.92636919 48971.14767998
 49522.5483048  51276.51797027 50051.96237318 47143.98927561
 52535.15723245 50521.64507908]
Mean: 50025.4385219577
Standard deviation: 1781.647390067774


In [24]:
from xgboost import XGBRegressor
xg_reg = XGBRegressor(learning_rate=0.08, 
                                 n_estimators=1300,
                                 max_depth=6,
                                 random_state=seed,
                                 gpu_id='0',
                                 booster="gbtree",
                                 warm_start = True,
                                 objective='reg:squarederror'
                                 ) 

xg_scores = cross_val_score(xg_reg, X_train, Y_train,
                             scoring="neg_mean_squared_error", cv=10)
xg_rmse_scores = np.sqrt(-xg_scores)
display_scores(xg_rmse_scores)

Scores: [44991.8077535  46228.63207701 48594.16721061 45305.60823664
 46195.47471453 45645.70148828 45690.11241333 44103.25746412
 51204.14302396 47507.3592668 ]
Mean: 46546.62636487592
Standard deviation: 1962.275229896435


In [25]:
xg_reg.fit(X_train, Y_train)
y_pred = xg_reg.predict(X_test)

np.sqrt(MSE(Y_test, y_pred))

46893.80851296068

###Classificação - Dataset Iris

In [26]:
from sklearn.datasets import load_iris
iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                        test_size=0.3, 
                                        random_state=seed)

In [27]:
from sklearn.tree import DecisionTreeClassifier
trclf = DecisionTreeClassifier(random_state=seed, max_depth=2)

trclf_scores = cross_val_score(trclf, x_train, y_train, cv=10, scoring='accuracy')
print(trclf_scores)

[0.90909091 1.         1.         0.72727273 0.81818182 1.
 1.         0.9        0.9        0.9       ]


In [28]:
trclf.fit(x_train, y_train)
y_pred = trclf.predict(x_test)

print(accuracy_score(y_test,y_pred))

0.9777777777777777


In [29]:
from xgboost import XGBClassifier
xgclf = XGBClassifier(random_state=seed, max_depth=2)

xgclf_scores = cross_val_score(xgclf, x_train, y_train, cv=10, scoring='accuracy')
print(xgclf_scores)

[0.90909091 1.         0.90909091 0.81818182 0.81818182 1.
 1.         1.         1.         0.9       ]


In [30]:
xgclf.fit(x_train, y_train)
y_pred = xgclf.predict(x_test)

print(accuracy_score(y_test,y_pred))

1.0
